<center>
<img src="./images/00_main_arcada.png" style="width:1400px">
</center>

## Lecture 2: Introduction to Container


## Instructor:
Truong An Pham (__An__)<br/> 
email:  truongan.pham@arcada.fi<br/>

# Goal for today

* Understand container.
* Be able to use docker for deploying DNNs with docker.


## Agenda 

* What is a container?
* Containerized application
* Container with database
* Containerized image
* Multi-Container Apps
* GPU-based container

# 1. Container

![docker_vm](images/docker_vs_vm_architecture.PNG)

* https://www.altexsoft.com/blog/docker-pros-and-cons/ 

## Kubernetes vs. Docker


<img style="float: center;" src="images/kubernets_docker.png" width=80%>







* https://www.atlassian.com/microservices/microservices-architecture/kubernetes-vs-docker











<img style="float: center;" src="images/ku_do_comparison.png" width=80%>

# Container orchestration overview

![docker_overview](images/docker_overview.PNG)

* https://www.spiceworks.com/tech/big-data/articles/what-is-docker/

# 2. Container application

We can play with docker via the following link

https://labs.play-with-docker.com/

In this lecture, I will ssh to a server and demonstrate the way of working with container

In [ ]:
# this is used for X11 forwarding to the server, then we can show the output of the server in your client machine in GUI.
ssh -X [SERVER]

## Try a containerized pytorch

In [ ]:
docker pull cnstark/pytorch:2.0.1-py3.10.11-ubuntu22.04

In [ ]:
# -it option is used for creating an interative driver for working with docker container
# --rm remove the container after exiting
docker run -it --rm cnstark/pytorch:2.0.1-py3.10.11-ubuntu22.04

In [ ]:
## Before, need to do apt-get udpate, and then apt-get install vim for installing text editor

## Install ultralytics: pip install -U ultralytics
## Install some dependencies on docker container: 
## apt install libgl1-mesa-glx
## apt-get install libglib2.0-0
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Images
imgs = ['https://ultralytics.com/images/zidane.jpg']  # batch of images

# Inference
results = model(imgs)

# Results
results.print()
results.save()  # or .show()

print(results.xyxy[0])  # img1 predictions (tensor), index 1 will print img2 predictions [tlX, tlY, brX, brY, conf, label]
results.pandas().xyxy[0] 

# 3. Container volume and bind mount

![docker_volume](images/docker_volume.webp)

In [ ]:
docker run -it --rm -v yolotestimages:/workspace/test_images [docker_container_name]

In [ ]:
## few images to test

# 4. Container image

In [ ]:
ARG BASE_IMAGE

FROM ${BASE_IMAGE}

ENV NVIDIA_DRIVER_CAPABILITIES ${NVIDIA_DRIVER_CAPABILITIES:-compute,utility}

RUN apt update && \
    apt install -y \
        wget build-essential zlib1g-dev libncurses5-dev libgdbm-dev libnss3-dev libssl-dev \
        libreadline-dev libffi-dev libsqlite3-dev libbz2-dev liblzma-dev && \
    apt clean && \
    rm -rf /var/lib/apt/lists/*

ARG PYTHON_VERSION

RUN cd /tmp && \
    wget https://www.python.org/ftp/python/${PYTHON_VERSION}/Python-${PYTHON_VERSION}.tgz && \
    tar -xvf Python-${PYTHON_VERSION}.tgz && \
    cd Python-${PYTHON_VERSION} && \
    ./configure --enable-optimizations && \
    make && make install && \
    cd .. && rm Python-${PYTHON_VERSION}.tgz && rm -r Python-${PYTHON_VERSION} && \
    ln -s /usr/local/bin/python3 /usr/local/bin/python && \
    ln -s /usr/local/bin/pip3 /usr/local/bin/pip && \
    python -m pip install --upgrade pip && \
    rm -r /root/.cache/pip

ARG PYTORCH_VERSION
ARG PYTORCH_VERSION_SUFFIX
ARG TORCHVISION_VERSION
ARG TORCHVISION_VERSION_SUFFIX
ARG TORCHAUDIO_VERSION
ARG TORCHAUDIO_VERSION_SUFFIX
ARG PYTORCH_DOWNLOAD_URL

RUN if [ ! $TORCHAUDIO_VERSION ]; \
    then \
        TORCHAUDIO=; \
    else \
        TORCHAUDIO=torchaudio==${TORCHAUDIO_VERSION}${TORCHAUDIO_VERSION_SUFFIX}; \
    fi && \
    if [ ! $PYTORCH_DOWNLOAD_URL ]; \
    then \
        pip install \
            torch==${PYTORCH_VERSION}${PYTORCH_VERSION_SUFFIX} \
            torchvision==${TORCHVISION_VERSION}${TORCHVISION_VERSION_SUFFIX} \
            ${TORCHAUDIO}; \
    else \
        pip install \
            torch==${PYTORCH_VERSION}${PYTORCH_VERSION_SUFFIX} \
            torchvision==${TORCHVISION_VERSION}${TORCHVISION_VERSION_SUFFIX} \
            ${TORCHAUDIO} \
            -f ${PYTORCH_DOWNLOAD_URL}; \
    fi && \
    rm -r /root/.cache/pip

WORKDIR /workspace

## A simpler Dockerfile for crawling images for testing

In [ ]:
FROM ubuntu:20.04
RUN apt-get update && apt-get install -y python3.9 python3.9-dev

WORKDIR /workspace

# 5. Multi-container apps

In [ ]:
## another docker 
FROM ubuntu:20.04
RUN apt-get update && apt-get install -y python3.9 python3.9-dev

WORKDIR /workspace1

In [ ]:
## docker compose .yaml file
## run with docker compose up
version: '3.8'
services:
  crawler:
    build: .
    volumes:
      - yovol:/workspace1/test_images
  yolo_detect:
    image: "yolocontainerdemo"
    volumes:
      - yovol:/workspace/test_images
volumes:
  yovol:

# 6. GPU-based container

![docker_gpu](images/nvidia-container.png)

In [ ]:
# https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/latest/install-guide.html
docker run --rm --runtime=nvidia --gpus all ubuntu nvidia-smi

# 7. Hands on exercises

Try to create a docker container to run yolov5 pretrained model to detect objects in at least 3 images.
Submit images plus, the output location of the prediction. 

# References:

1. http://ip172-18-0-52-cng2vl8l2o9000co9t70-80.direct.labs.play-with-docker.com/tutorial/sharing-our-app/
2. https://docs.nvidia.com/deeplearning/frameworks/user-guide/index.html